## Imports

In [26]:
#| export
import torch
from IPython.core.magic import register_cell_magic
from IPython.display import HTML, display
import html
import math
import copy
import numpy as np

https://arxiv.org/pdf/1706.03762

I will be doing an implementation of the seminal paper "Attention Is All You Need."

## Transformer Architecture
![pasted_image_815012f1-e052-4514-a18c-db31f235d612.png](attachment:815012f1-e052-4514-a18c-db31f235d612)

### Architecture Details


![pasted_image_670cb52c-fedb-4185-bb81-749fd2bffd33.png](attachment:670cb52c-fedb-4185-bb81-749fd2bffd33)

## Attention

Implement the scaled dot-product attention mechanism. Remember the formula: Attention(Q,K,V) = softmax(QK^T / √d_k)V

In [27]:
#| export
def attention(Q, K, V, dropout, mask):
    """Scaled Dot-Product Attention"""
    sqrt_d_k = math.sqrt(K.size(-1))
    scores = Q @ K.transpose(-2,-1) / sqrt_d_k
    
    if mask is not None:
        scores = scores.masked_fill(mask==0, -1e9)
    
    attention_weights = torch.softmax(scores, dim=-1)
    
    if dropout is not None:
        attention_weights = dropout(attention_weights)
    
    output = attention_weights @ V
    
    return output

#### Unit Test

In [28]:
# Test attention
Q = torch.randn(2, 1, 4, 8)
K = torch.randn(2, 1, 4, 8)
V = torch.randn(2, 1, 4, 8)
result = attention(Q, K, V, None, None)
print("Result shape:", result.shape)

Result shape: torch.Size([2, 1, 4, 8])


## MultiHeadedAttention

Implement multi-head attention. Split d_model into h heads, apply attention to each, then concatenate.

In [29]:
#| export
class MultiHeadedAttention(torch.nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0
        
        self.d_k = d_model // h
        self.h = h
        self.dropout = torch.nn.Dropout(p=dropout)
        self.attn = None
        
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(d_model, d_model) for x in range(4)])
    
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        Q = self.linear_layers[0](query)
        K = self.linear_layers[1](key)
        V = self.linear_layers[2](value)
        
        Q = Q.reshape(batch_size, -1, self.h, self.d_k).transpose(1,2)
        K = K.reshape(batch_size, -1, self.h, self.d_k).transpose(1,2)
        V = V.reshape(batch_size, -1, self.h, self.d_k).transpose(1,2)
        
        if mask is not None:
            mask = mask.unsqueeze(1)
        
        x = attention(Q, K, V, self.dropout, mask)
        
        x = x.transpose(1,2).reshape(batch_size, -1, self.h * self.d_k)
        
        output = self.linear_layers[3](x)
        
        return output

#### Unit Test

In [30]:
# Test MultiHeadedAttention
mha = MultiHeadedAttention(h=8, d_model=512, dropout=0.1)
x = torch.randn(2, 10, 512)
output = mha(x, x, x)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10, 512]) Output shape: torch.Size([2, 10, 512])


## PositionwiseFeedForward

Implement the position-wise feed-forward network: FFN(x) = max(0, xW1 + b1)W2 + b2

In [31]:
#| export
class PositionwiseFeedForward(torch.nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear_layer = torch.nn.Linear(d_model, d_ff)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=dropout)
        self.output_layer = torch.nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.linear_layer(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.output_layer(x)
        return x

#### Unit Test

In [32]:
# Test PositionwiseFeedForward
ffn = PositionwiseFeedForward(d_model=512, d_ff=2048, dropout=0.1)
x = torch.randn(2, 10, 512)
output = ffn(x)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10, 512]) Output shape: torch.Size([2, 10, 512])


## Positional Encoding

Since attention has no notion of position, we add positional encodings using sin/cos functions of different frequencies.

In [33]:
#| export
class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        even_indices = torch.arange(0, d_model, 2)
        
        position = torch.arange(0, max_len).unsqueeze(1)
        
        div_term = torch.exp(-even_indices * (torch.log(torch.tensor(10000.0)) / d_model))
        
        pe[:, ::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        seq_len = x.size(1)
        x = x + self.pe[:,:seq_len,:].requires_grad_(False)
        return self.dropout(x)

#### Unit Test

In [34]:
# Test PositionalEncoding
pe = PositionalEncoding(d_model=512, dropout=0.1)
x = torch.zeros(2, 10, 512)
output = pe(x)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10, 512]) Output shape: torch.Size([2, 10, 512])


## Encoder Layer

Each encoder layer has two sub-layers: multi-head self-attention and feed-forward network, each with residual connections.

### Sublayer Connection

In [35]:
#| export
class SublayerConnection(torch.nn.Module):
    "A residual connection followed by layer norm"
    def __init__(self, size, dropout):
        super().__init__()
        self.layer_norm = torch.nn.LayerNorm(size)
        self.dropout = torch.nn.Dropout(dropout)
        
    def forward(self, x, sublayer):
        y = self.layer_norm(x)
        y = sublayer(y)
        y = self.dropout(y)
        return x + y

### Final clones function

In [36]:
#| export
def clones(module, N):
    "Produce N identical layers"
    return torch.nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

### Final EncoderLayer Function

In [37]:
#| export
class EncoderLayer(torch.nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.size = size
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = torch.nn.ModuleList([SublayerConnection(size, dropout) for _ in range(2)])
    
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        x = self.sublayer[1](x, lambda x: self.feed_forward(x))
        return x

#### Unit Test

In [38]:
# Test EncoderLayer
attn = MultiHeadedAttention(h=8, d_model=512, dropout=0.1)
ff = PositionwiseFeedForward(d_model=512, d_ff=2048, dropout=0.1)
encoder_layer = EncoderLayer(size=512, self_attn=attn, feed_forward=ff, dropout=0.1)
x = torch.randn(2, 10, 512)
output = encoder_layer(x, None)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10, 512]) Output shape: torch.Size([2, 10, 512])


## Encoder Class

Stack N encoder layers together.

In [39]:
#| export
class Encoder(torch.nn.Module):
    "Stack of N encoder layers"
    def __init__(self, layer, N):
        super().__init__()
        self.encoders = clones(layer, N)
        self.layer_norm = torch.nn.LayerNorm(layer.size)
    
    def forward(self, x, mask):
        for encoder in self.encoders:
            x = encoder(x, mask)
        return self.layer_norm(x)

#### Unit test

In [40]:
# Test Encoder
encoder = Encoder(encoder_layer, N=6)
x = torch.randn(2, 10, 512)
output = encoder(x, None)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10, 512]) Output shape: torch.Size([2, 10, 512])


## Decoder Layer

Each decoder layer has three sub-layers: masked self-attention, cross-attention to encoder output, and feed-forward.

In [41]:
#| export
class DecoderLayer(torch.nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super().__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayers = clones(SublayerConnection(size, dropout), 3)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        x = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayers[1](x, lambda x: self.src_attn(x, memory, memory, src_mask))
        x = self.sublayers[2](x, self.feed_forward)
        return x

#### Unit test

In [42]:
# Test DecoderLayer
attn1 = MultiHeadedAttention(h=8, d_model=512, dropout=0.1)
attn2 = MultiHeadedAttention(h=8, d_model=512, dropout=0.1)
ff = PositionwiseFeedForward(d_model=512, d_ff=2048, dropout=0.1)
decoder_layer = DecoderLayer(size=512, self_attn=attn1, src_attn=attn2, feed_forward=ff, dropout=0.1)
x = torch.randn(2, 10, 512)
memory = torch.randn(2, 10, 512)
output = decoder_layer(x, memory, None, None)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10, 512]) Output shape: torch.Size([2, 10, 512])


## Decoder

Stack N decoder layers together.

In [43]:
#| export
class Decoder(torch.nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.layer_norm = torch.nn.LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.layer_norm(x)

### Unit test

In [44]:
# Test Decoder
decoder = Decoder(decoder_layer, N=6)
x = torch.randn(2, 10, 512)
memory = torch.randn(2, 10, 512)
output = decoder(x, memory, None, None)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10, 512]) Output shape: torch.Size([2, 10, 512])


## Embeddings

Convert token IDs to dense vectors, scaled by sqrt(d_model).

In [45]:
#| export
class Embeddings(torch.nn.Module):
    def __init__(self, d_model, vocab):
        super().__init__()
        self.d_model = d_model
        self.embedding = torch.nn.Embedding(vocab, d_model)
        
    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

### Unit test

In [46]:
# Test Embeddings
emb = Embeddings(d_model=512, vocab=1000)
x = torch.randint(0, 1000, (2, 10))
output = emb(x)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10]) Output shape: torch.Size([2, 10, 512])


## Generator class

Final linear layer + log softmax to convert decoder output to token probabilities.

In [47]:
#| export
class Generator(torch.nn.Module):
    def __init__(self, d_model, vocab):
        super().__init__()
        self.linear = torch.nn.Linear(d_model, vocab)
        self.logsoftmax = torch.nn.LogSoftmax(dim=-1)
        
    def forward(self, x):
        x = self.linear(x)
        return self.logsoftmax(x)

#### Unit test

In [48]:
# Test Generator
gen = Generator(d_model=512, vocab=1000)
x = torch.randn(2, 10, 512)
output = gen(x)
print("Input shape:", x.shape, "Output shape:", output.shape)

Input shape: torch.Size([2, 10, 512]) Output shape: torch.Size([2, 10, 1000])


### Final Encoder Decoder Function.

In [49]:
#| export
class EncoderDecoder(torch.nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    
    def encode(self, src, src_mask):
        x = self.src_embed(src)
        return self.encoder(x, src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        x = self.tgt_embed(tgt)
        return self.decoder(x, memory, src_mask, tgt_mask)
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        memory = self.encode(src, src_mask)
        return self.decode(memory, src_mask, tgt, tgt_mask)

## Make Model

Helper function to construct the full transformer model.

In [50]:
#| export
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Construct transformer model from hyperparameters"
    multi_head_attentions = [copy.deepcopy(MultiHeadedAttention(h, d_model, dropout)) for _ in range(3)]
    feedforward_layers = [copy.deepcopy(PositionwiseFeedForward(d_model, d_ff, dropout)) for _ in range(2)]
    positional_encoding = [PositionalEncoding(d_model, dropout) for _ in range(2)]
    
    src_embedding_layer = Embeddings(d_model, src_vocab)
    tgt_embeddings_layer = Embeddings(d_model, tgt_vocab)
    src_embed = torch.nn.Sequential(src_embedding_layer, positional_encoding[0])
    tgt_embed = torch.nn.Sequential(tgt_embeddings_layer, positional_encoding[1])
    
    encoder_layer = EncoderLayer(d_model, multi_head_attentions[0], feedforward_layers[0], dropout)
    decoder_layer = DecoderLayer(d_model, multi_head_attentions[1], multi_head_attentions[2], feedforward_layers[1], dropout)
    
    encoder = Encoder(encoder_layer, N)
    decoder = Decoder(decoder_layer, N)
    generator = Generator(d_model, tgt_vocab)
    
    encoder_decoder = EncoderDecoder(encoder, decoder, src_embed, tgt_embed, generator)
    
    for p in encoder_decoder.parameters():
        if p.dim() > 1:
            torch.nn.init.xavier_uniform_(p)
    
    return encoder_decoder

## Copy Code Task

Training utilities for the copy task (provided complete).

### Final subsequent mask function

In [51]:
#| export
def subsequent_mask(size):
    "Create mask to prevent attention to future positions"
    lower_t = torch.ones([size, size]).tril().bool()
    lower_t = lower_t.unsqueeze(0)
    return lower_t

### Final Batch Class for copy example

In [52]:
#| export
class Batch:
    def __init__(self, src, tgt=None, pad=2):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:,1:]
            self.pad_tgt_mask = (self.tgt!=pad).unsqueeze(-2)
            self.subseq_tgt_mask = subsequent_mask(self.tgt.size(1))
            self.tgt_mask = self.pad_tgt_mask & self.subseq_tgt_mask
            self.ntokens = (self.tgt_y!=pad).sum()

### Final Data Gen Function

In [53]:
#| export
def data_gen(V, batch_size, nbatches):
    "Generate random data for a src-tgt copy task"
    for i in range(nbatches):
        random_int = np.random.randint(1, V, size=[batch_size, 10])
        random_int[:,0] = 1
        random_int = torch.tensor(random_int)
        src = random_int
        tgt = random_int
        yield Batch(src, tgt, 0)

### Final Simple Loss Compute Function

In [54]:
#| export
class SimpleLossCompute:
    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion
        
    def __call__(self, x, y, norm):
        pred = self.generator(x)
        vocab = pred.size(-1)
        pred_flat = pred.reshape(-1, vocab)
        y_flat = y.reshape(-1)
        loss = self.criterion(pred_flat, y_flat) / norm
        return loss.data * norm, loss

### Final Run Epoch Function for Copy

In [55]:
#| export
def run_epoch(data_iter, model, loss_compute, optimizer, scheduler, mode="train"):
    total_tokens = 0
    total_loss = 0
    
    for i, batch in enumerate(data_iter):
        pred = model(batch.src, batch.tgt, batch.src_mask, batch.tgt_mask)
        num_loss, tensor_loss = loss_compute(pred, batch.tgt_y, batch.ntokens)
        if mode == "train":
            tensor_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
        total_loss += num_loss 
        total_tokens += batch.ntokens
    
    return total_loss / total_tokens

### Greedy Decoding

In [56]:
#| export
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.LongTensor([[start_symbol]])
    for _ in range(max_len - 1):
        tgt_mask = subsequent_mask(ys.size(1))
        x = model.decode(memory, src_mask, ys, tgt_mask)
        pred = model.generator(x[:, -1])
        _, max_indices = torch.max(pred, dim=-1)
        max_indices = max_indices.data[0]
        ys = torch.cat([ys, torch.ones(1,1).type_as(src.data).fill_(max_indices)], dim=-1)
    return ys

### Training Example

In [60]:
# Create small model for testing
V = 11
criterion = torch.nn.CrossEntropyLoss()
model = make_model(V, V, N=2, d_model=64, d_ff=128, h=4, dropout=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9)

from torch.optim.lr_scheduler import LambdaLR
def rate(step, model_size=64, factor=1.0, warmup=400):
    if step == 0:
        step = 1
    return factor * (model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5)))

scheduler = LambdaLR(optimizer, lr_lambda=lambda step: rate(step))

print("Training...")
for epoch in range(30):
    model.train()
    loss_compute = SimpleLossCompute(model.generator, criterion)
    loss = run_epoch(data_gen(V, batch_size=30, nbatches=20), model, loss_compute, optimizer, scheduler, mode="train")
    print(f"Epoch {epoch} Loss: {loss:.4f}")

print("\nTraining complete!")

Training...
Epoch 0 Loss: 0.0109
Epoch 1 Loss: 0.0094
Epoch 2 Loss: 0.0085
Epoch 3 Loss: 0.0079
Epoch 4 Loss: 0.0073
Epoch 5 Loss: 0.0070
Epoch 6 Loss: 0.0066
Epoch 7 Loss: 0.0060
Epoch 8 Loss: 0.0054
Epoch 9 Loss: 0.0050
Epoch 10 Loss: 0.0044
Epoch 11 Loss: 0.0038
Epoch 12 Loss: 0.0033
Epoch 13 Loss: 0.0028
Epoch 14 Loss: 0.0024
Epoch 15 Loss: 0.0022
Epoch 16 Loss: 0.0017
Epoch 17 Loss: 0.0015
Epoch 18 Loss: 0.0013
Epoch 19 Loss: 0.0012
Epoch 20 Loss: 0.0010
Epoch 21 Loss: 0.0008
Epoch 22 Loss: 0.0008
Epoch 23 Loss: 0.0007
Epoch 24 Loss: 0.0007
Epoch 25 Loss: 0.0006
Epoch 26 Loss: 0.0007
Epoch 27 Loss: 0.0005
Epoch 28 Loss: 0.0004
Epoch 29 Loss: 0.0005

Training complete!


### Test Inference

In [62]:
model.eval()
src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
src_mask = torch.ones(1, 1, 10)

print("Source:", src)
result = greedy_decode(model, src, src_mask, max_len=10, start_symbol=1)
print("Generated:", result)
print("\nSuccess!", torch.equal(src, result))

Source: tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])
Generated: tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])

Success! True


## 🎉 Congratulations! 🎉

You've implemented a Transformer from scratch and trained it successfully!

### Resources:
- [Original Paper](https://arxiv.org/pdf/1706.03762)
- [Harvard NLP Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)